# Task 0: Understanding the data

## 1. Load libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## 2. Load data

In [2]:
journeys = pd.read_csv('../data/journeys.csv')
utilization = pd.read_csv('../data/utilization.csv', sep=';', header=1)

In [3]:
journeys.head(3)

,Trip ID Hash,Car ID Hash,Car Parking Address Postcode,Car Parking Address City,Trip Start At Local Time,Trip End At Local Time,Trip Created At Local Time,Trip Sum Trip Price
0,00010247034d28272cf5e1e16b43f52f,9ddac6a5fb0b3962db3b2f42fd31d3f9,94110,San Francisco,2017-11-14 22:15:00,2017-11-15 02:15:00,2017-11-14 20:32:59,$20.00
1,0001556e5d28e66b3318e89c094186c4,0f386ec90e0710ca143922520eaf8a3e,94117,San Francisco,2017-12-07 17:00:00,2017-12-08 04:00:00,2017-12-07 04:51:42,$55.00
2,000286e8771651e6983555c62558b57f,63e4509c686d891f5364c24339369460,94115,San Francisco,2017-03-23 10:00:00,2017-03-23 12:00:00,2017-03-23 08:49:48,$18.00


In [4]:
journeys.dtypes

Trip ID Hash                    object
Car ID Hash                     object
Car Parking Address Postcode     int64
Car Parking Address City        object
Trip Start At Local Time        object
Trip End At Local Time          object
Trip Created At Local Time      object
Trip Sum Trip Price             object
dtype: object

In [5]:
journeys.shape

(56537, 8)

In [6]:
journeys.isna().sum()

Trip ID Hash                    0
Car ID Hash                     0
Car Parking Address Postcode    0
Car Parking Address City        0
Trip Start At Local Time        0
Trip End At Local Time          0
Trip Created At Local Time      0
Trip Sum Trip Price             0
dtype: int64

In [7]:
utilization.head(3)

,Car ID Hash,Car Hourly Utilization Aggregated At Time,Car Hourly Utilization Sum Available Minutes,Car Hourly Utilization Sum Utilized Minutes
0,001469b449411d52206f2dc5e523664d,2017-11-30 15:00:00,0.0,0.0
1,001469b449411d52206f2dc5e523664d,2017-11-30 16:00:00,0.0,0.0
2,001469b449411d52206f2dc5e523664d,2017-11-30 17:00:00,0.0,0.0


In [8]:
utilization.dtypes

Car ID Hash                                      object
Car Hourly Utilization Aggregated At Time        object
Car Hourly Utilization Sum Available Minutes    float64
Car Hourly Utilization Sum Utilized Minutes     float64
dtype: object

In [9]:
utilization.shape

(999999, 4)

In [10]:
utilization.isna().sum()

Car ID Hash                                     0
Car Hourly Utilization Aggregated At Time       0
Car Hourly Utilization Sum Available Minutes    0
Car Hourly Utilization Sum Utilized Minutes     0
dtype: int64

## 3. Data Cleansing

In [11]:
for col in [col for col in utilization.columns if 'Time' in col]:
    utilization[col] = pd.to_datetime(utilization[col])

In [12]:
for col in [col for col in journeys.columns if 'Time' in col]:
    journeys[col] = pd.to_datetime(journeys[col])

In [13]:
utilization.columns = [col.lower().replace(' ', '_') for col in utilization.columns]
journeys.columns = [col.lower().replace(' ', '_') for col in journeys.columns]

In [14]:
utilization.dtypes

car_id_hash                                             object
car_hourly_utilization_aggregated_at_time       datetime64[ns]
car_hourly_utilization_sum_available_minutes           float64
car_hourly_utilization_sum_utilized_minutes            float64
dtype: object

In [15]:
journeys.dtypes

trip_id_hash                            object
car_id_hash                             object
car_parking_address_postcode             int64
car_parking_address_city                object
trip_start_at_local_time        datetime64[ns]
trip_end_at_local_time          datetime64[ns]
trip_created_at_local_time      datetime64[ns]
trip_sum_trip_price                     object
dtype: object

In [16]:
journeys['trip_sum_trip_price'] = (
    journeys['trip_sum_trip_price'].str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

## 4. Creating new features

In [17]:
journeys['duration'] = (
    journeys['trip_end_at_local_time'] - journeys['trip_start_at_local_time']
)

journeys['duration'] = journeys['duration'].astype('timedelta64[s]')

In [18]:
journeys['creation_to_start'] = (
    journeys['trip_start_at_local_time'] - journeys['trip_created_at_local_time']
)
journeys['creation_to_start'] = journeys['creation_to_start'].astype('timedelta64[s]')

In [19]:
journeys[['duration', 'creation_to_start', 'trip_sum_trip_price']].describe()

,duration,creation_to_start,trip_sum_trip_price
count,5.653700e+04,5.653700e+04,56537.000000
mean,3.612792e+04,6.088025e+04,53.790023
std,6.021352e+04,1.598641e+05,61.664701
min,3.600000e+03,6.000000e+00,3.030000
25%,1.260000e+04,1.128000e+03,24.000000
50%,2.160000e+04,9.909000e+03,39.870000
75%,3.510000e+04,5.754400e+04,63.250000
max,3.715200e+06,2.418952e+06,3225.000000


In [20]:
journeys['trip_start_hour'] = journeys['trip_start_at_local_time'].dt.hour
journeys['trip_end_hour'] = journeys['trip_end_at_local_time'].dt.hour
journeys['trip_start_day'] = journeys['trip_start_at_local_time'].dt.day
journeys['trip_start_month'] = journeys['trip_start_at_local_time'].dt.month
journeys['trip_start_dow'] = journeys['trip_start_at_local_time'].dt.dayofweek

In [21]:
journeys['trip_start_dow'] = journeys['trip_start_dow'].map(
    {
        0: 'Mon',
        1: 'Tue',
        2: 'Wen',
        3: 'Thu',
        4: 'Fri',
        5: 'Sat',
        6: 'Sun'
    }
)